In [53]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn import preprocessing

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader

from format_data import read_dataset, aggregate_dataset, clean_dataset, sliding_window
from dataset import MoodDataset

In [54]:
dataset = pd.read_csv('data/dataset_mood_smartphone.csv')
dataset = read_dataset(dataset)
dataset = aggregate_dataset(dataset)

# Remove rows without mood
dataset = dataset[dataset['mood'].notnull()]

# Fill NaN cells with average column value 
dataset = dataset.fillna(dataset.mean())

# Remove columns 
drop_columns = ['appCat.builtin', 
                'appCat.finance', 'appCat.office', 'appCat.social', 
                'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather']

dataset = dataset.drop(columns=drop_columns)

stored_ids = dataset['id']
stored_dates = dataset['date']
stored_moods = dataset['mood']

dataset = dataset.drop(['id', 'date', 'mood'], axis=1)
stored_columns = dataset.columns

# dataset = dataset.values #returns a numpy array
ss = preprocessing.StandardScaler()
dataset = ss.fit_transform(dataset)
dataset = pd.DataFrame(dataset, columns=stored_columns)

dataset.insert(0, 'id', list(stored_ids), allow_duplicates=True)
dataset.insert(1, 'date', list(stored_dates), allow_duplicates=True)
dataset.insert(2, 'mood', list(stored_moods), allow_duplicates=True)

display(dataset)

samples = sliding_window(dataset)

/var/folders/jr/91tgzxq96b30vd94j032zrwc0000gn/T/ipykernel_8575/260059517.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dataset = dataset.fillna(dataset.mean())


variable,id,date,mood,activity,appCat.communication,appCat.entertainment,appCat.game,appCat.other,call,circumplex.arousal,circumplex.valence,screen,sms
0,AS14.01,2014-02-26,6.0,3.012761e-16,-1.748233e-15,1.451039e-16,2.585099e-16,-2.299761e-16,-1.061799e+00,-0.220617,0.144875,-2.022333e-16,-3.271966e-01
1,AS14.01,2014-02-27,6.0,3.012761e-16,-1.748233e-15,1.451039e-16,2.585099e-16,-2.299761e-16,1.611708e-16,0.156768,-0.814992,-2.022333e-16,3.125141e-16
2,AS14.01,2014-03-21,6.0,1.620330e-01,1.514142e+00,-1.360081e-01,2.585099e-16,1.671725e-01,7.528228e-01,0.458676,-1.122150,2.645451e+00,3.125141e-16
3,AS14.01,2014-03-22,6.0,1.278214e+00,9.511621e-01,-7.193831e-01,2.585099e-16,-2.147709e-01,-3.359503e-01,1.062492,-0.431045,1.345713e-02,-1.030916e+00
4,AS14.01,2014-03-23,7.0,2.563706e-01,1.068374e+00,-7.187309e-01,2.585099e-16,-2.830637e-01,1.611708e-16,0.458676,0.260059,1.539141e-01,3.125141e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1263,AS14.33,2014-05-27,6.0,-1.155130e+00,-2.809764e-01,-6.993636e-01,2.585099e-16,-9.463005e-02,-1.061799e+00,-0.748957,-0.661414,-4.428839e-01,-3.271966e-01
1264,AS14.33,2014-05-28,8.0,-1.717349e-01,-3.416275e-01,-3.866508e-01,2.585099e-16,9.498830e-01,2.204520e+00,0.156768,1.181531,1.832057e+00,-1.030916e+00
1265,AS14.33,2014-05-29,7.0,5.452481e-01,-9.829213e-01,-6.916774e-01,2.585099e-16,-4.007177e-01,3.898985e-01,-1.855953,0.720795,-5.586296e-01,-1.030916e+00
1266,AS14.33,2014-05-30,7.0,8.008389e-01,-4.550686e-01,-6.128745e-01,2.585099e-16,-3.375820e-01,2.697411e-02,-1.050865,-2.504358,7.595758e-01,3.125141e-16


In [56]:
dataset.var(ddof=0)

/var/folders/jr/91tgzxq96b30vd94j032zrwc0000gn/T/ipykernel_8575/3400278623.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dataset.var(ddof=0)


variable
mood                    0.655312
activity                1.000000
appCat.communication    1.000000
appCat.entertainment    1.000000
appCat.game             1.000000
appCat.other            1.000000
call                    1.000000
circumplex.arousal      1.000000
circumplex.valence      1.000000
screen                  1.000000
sms                     1.000000
dtype: float64

In [57]:
X = []
y = []

for index, sample in enumerate(samples):
    X.append([])
    for day_features in sample[0]:
        X[index].append(np.array(day_features))
    X[index] = np.array(X[index])
    y.append(sample[1])    
    
X = np.array(X)
y = np.array(y)
print(np.shape(X))
print(np.shape(y))

(830, 5, 11)
(830,)


In [60]:
# Testing fake data to see if model makes sense

# X = [np.array([1]) for x in range(100)]
# X += [np.array([2]) for y in range(100)]
# X = np.array(X)

# y = [1 for x in range(100)]
# y += [2 for y in range(100)]
# y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

train_dataset = MoodDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=50, shuffle=True)
test_dataset = MoodDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=50, shuffle=True)

In [63]:
class LSTM(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, output_size) :
        super().__init__()
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, output_size)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, dropout):
        if dropout:
            x = self.dropout(x)
        else:
            pass
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [ ]:
num_epochs = 10000
learning_rate = 0.0001

embedding_dim = 11
hidden_size = 20
output_size = 10
lstm = LSTM(embedding_dim, hidden_size, output_size)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for step, (x, y) in enumerate(train_dataloader): 
        
        outputs = lstm(x, True)

        optimizer.zero_grad()

        # obtain the loss function
        loss = criterion(outputs, y.float())

        loss.backward()

        optimizer.step()
    if epoch % 100 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))
        
        correct = 0
        total = 0
        # since we're not training, we don't need to calculate the gradients for our outputs
        with torch.no_grad():
            for step, (x, y) in enumerate(test_dataloader): 

                # calculate outputs by running images through the network
                outputs = lstm(x, False)
                # the class with the highest energy is what we choose as prediction
                predicted = torch.argmax(outputs, dim=1)
                y = torch.argmax(y, dim=1)
                total += y.size(0)
                correct += (predicted == y).sum().item()
                
        print(f'Accuracy of the network on the test data: {100 * correct // total}%')

Epoch: 0, loss: 0.11881
Accuracy of the network on the test data: 49%
Epoch: 100, loss: 0.07561
Accuracy of the network on the test data: 53%
Epoch: 200, loss: 0.06322
Accuracy of the network on the test data: 56%
Epoch: 300, loss: 0.06899
Accuracy of the network on the test data: 55%
Epoch: 400, loss: 0.05945
Accuracy of the network on the test data: 56%
Epoch: 500, loss: 0.05686
Accuracy of the network on the test data: 57%
Epoch: 600, loss: 0.05646
Accuracy of the network on the test data: 56%
Epoch: 700, loss: 0.06370
Accuracy of the network on the test data: 56%
Epoch: 800, loss: 0.05548
Accuracy of the network on the test data: 55%
Epoch: 900, loss: 0.07862
Accuracy of the network on the test data: 55%
Epoch: 1000, loss: 0.03597
Accuracy of the network on the test data: 56%
Epoch: 1100, loss: 0.08729
Accuracy of the network on the test data: 54%
Epoch: 1200, loss: 0.07992
Accuracy of the network on the test data: 56%
Epoch: 1300, loss: 0.04528
Accuracy of the network on the test 

In [49]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for step, (x, y) in enumerate(test_dataloader): 
        
        # calculate outputs by running images through the network
        outputs = lstm(x, False)
        # the class with the highest energy is what we choose as prediction
        predicted = torch.argmax(outputs, dim=1)
        y = torch.argmax(y, dim=1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

print(f'Accuracy of the network on the test data: {100 * correct // total}%')

Accuracy of the network on the test data: 25%
